In [ ]:
!pip install --upgrade transformers
!pip install --upgrade evaluate
!pip install --upgrade datasets

import transformers
import datasets
import evaluate

print(transformers.__version__)
print(datasets.__version__)
print(evaluate.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 85.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are inst

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:
raw_data= load_dataset("imdb")
raw_data

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
model_name= "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation= True)

In [ ]:
tokenized_data = raw_data.map(tokenize_function, batched= True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
tokenized_data=tokenized_data.remove_columns(['text']).rename_column('label', 'labels').with_format('pt')
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [ ]:
train_args = TrainingArguments("bert-sentiment-analysis",
                               eval_strategy= 'epoch',
                               num_train_epochs = 2,
                               learning_rate = 5e-5,
                               weight_decay = 0.005,
                               per_device_train_batch_size = 8,
                               per_device_eval_batch_size = 8,
                               save_strategy= 'epoch',
                               save_total_limit = 2,
                               logging_steps=50,
                               metric_for_best_model='accuracy',
                               report_to='none')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(pred):
    metric = evaluate.load("accuracy")
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    return metric.compute(predictions=preds, references=labels)

In [ ]:
trainer = Trainer(
    model,
    args= train_args,
    train_dataset= tokenized_data["train"],
    eval_dataset= tokenized_data["test"],
    data_collator= data_collator,
    tokenizer= tokenizer,
    compute_metrics= compute_metrics
)

<ipython-input-11-29675d88d140>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.305800,0.331705,0.912520
2,0.196700,0.276809,0.937360


TrainOutput(global_step=6250, training_loss=0.249313664894104, metrics={'train_runtime': 5960.2054, 'train_samples_per_second': 8.389, 'train_steps_per_second': 1.049, 'total_flos': 1.240048981688064e+16, 'train_loss': 0.249313664894104, 'epoch': 2.0})

In [ ]:

test_data = tokenized_data["test"].remove_columns(['labels'])
test_data

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})

In [ ]:
preds= trainer.predict(test_data)
preds

PredictionOutput(predictions=array([[ 3.5374904, -3.4278224],
       [ 2.527215 , -2.5758944],
       [ 3.5640213, -3.4598737],
       ...,
       [-2.0318289,  1.9685686],
       [-2.0869033,  1.9958346],
       [-3.0702603,  2.9568472]], dtype=float32), label_ids=None, metrics={'test_runtime': 680.6487, 'test_samples_per_second': 36.73, 'test_steps_per_second': 4.591})

In [ ]:
predicts= np.argmax(preds.predictions, axis=-1)
predicts

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
metric = evaluate.load("accuracy")
acc = metric.compute(predictions=predicts, references=tokenized_data["test"]["labels"])

In [ ]:
acc

{'accuracy': 0.93736}

In [ ]:
import shutil

shutil.make_archive('bert-sentiment-analysis', 'zip', 'bert-sentiment-analysis/checkpoint-6250')

'/content/bert-sentiment-analysis.zip'

In [ ]:
from google.colab import files
files.download('bert-sentiment-analysis.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>